<a href="https://colab.research.google.com/github/purvasingh96/pytorch-examples/blob/master/Basics/03.%20Neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks

## Defining Neural Network
* Neural networks can be constructed using the `torch.nn` package.
* An `nn.Module` contains layers, and a method `forward(input)` that returns the `output.`

## Training Neural Network
A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`



In [0]:
# Network Definition
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)
    self.fc1 = nn.Linear(16 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    # before sending input to linear layers
    # change the tensor's shape accordingly  
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:] # all dimensions except batch dimension
    num_features = 1
    for s in size:
      num_features *= s
    return num_features


In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Key points - I

* We only need to define `forward` function.
* `backward` function gets automatically defined using `autograd`
* Learnable parameters of model are returned by `net.parameters()`

In [7]:
params = list(net.parameters())
print(len(params))
# conv1's weights
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


# Key points - II
* `torch.nn` supports only mini-batches.
* `nn.Conv2d` will take 4D Tensor of `nSamples x nChannels x Height x Width.`
* In case of single sample use `input.unsqueeze(0)` to add fake mini-batch.

# Recap - I

| PyTorch Module      | Comments                                                                              |
|---------------------|---------------------------------------------------------------------------------------|
| `torch.Tensor`      | multi-dimensional array supporting functions such as `backward`                       |
| `nn.Module`         | Encapsulates parameters to move them to GPU etc.                                      |
| `nn.Parameter`      | automatically registered as a parameter when assigned as an attribute to a  `Module`. |
| `autograd.Function` | Implements `forward()` and `backward()` functions of neural network.                  |